In [1]:
from __future__ import print_function
import sys, os
import time
import pickle
import rospy
import numpy as np
from scipy import signal
import scipy
from sensor_msgs.msg import Image
from cv_bridge import CvBridge, CvBridgeError
import cv2

from termcolor import colored
from enum import Enum

import matplotlib.pyplot as plt
import matplotlib.patches as patches

# wrapper/helper functions
sys.path.insert(0, '/home/torobo/catkin_ws/src/torobo_robot/torobo_rnn/scripts')
sys.path.insert(0, '/home/torobo/catkin_ws/src/tutorial/PyTorch-YOLOv3')

from torobo_rnn_utils__upd3 import *
from torobo_driver import torobo_easy_command

from detect_upd import Recog, Recog2


class NewRec(threading.Thread):

    """ RECORDS A DATASET FOR THE RNN """
    
    def __init__(self):
        threading.Thread.__init__(self)
        self.recording = False
        self.step = 0.1
        self.data = {}
        self.fname = "DS.dat"
    
    def run(self):
        if self.recording == True:
            self.recording = False
            print('Killing the previous recorder')
            time.sleep(1.5)
        self.recording = True
        self.record()
    
    def stop(self):
        self.recording = False
        time.sleep(1)
        print('Recording is {}'.format('ON' if self.is_alive() else 'OFF'))
        
        self.data['img'] = np.stack(self.data['img'])
        self.data['joints'] = np.stack(self.data['joints'])
        self.data['t'] = np.stack(self.data['t'])
        with open(self.fname, 'wb') as f:
            pickle.dump(self.data, f)
        print('Data saved')

        
    def record(self):
        global torobo, imagebuff
        print('Recording is {}'.format('ON' if self.is_alive() else 'OFF'))
        # overwrite
        if "DS.dat" in os.listdir("."):
            open("DS.dat", 'w').close()
        t = 0
        start_t = time.time()
        self.data['joints'] = []
        self.data['img'] = []
        self.data['t'] = []
        while self.recording:
            self.data['joints'].append(np.radians(get_cur_joints(torobo)[0]))
            self.data['img'].append(cv2.resize(imagebuff, (64,64), interpolation = cv2.INTER_AREA) )
            self.data['t'].append(time.time()-start_t)
            time.sleep(self.step)
            t += self.step

class Tracker(threading.Thread):
    """ - LOADS THE YOLO MODEL
        - FORWARD PASSES THE FRAME THROUGH THE MODEL
        - STORES BOUNDING BOX COORDINATES IN SELF.
    """
    
    def __init__(self):
        threading.Thread.__init__(self)
        
        checkpoint = 42
        self.recog = Recog2(checkpoint)
        self.box_cener = (0.0, 0.0)
        self.x1, self.y1, self.x2, self.y2 = 0.0, 0.0, 0.0, 0.0
        self.new_hor = 0
        self.new_ver = 0
        self.Norm = scipy.stats.norm(0, 20)
        self.scale = self.Norm.pdf(0)
        
        self.keepgoing = False

    def stop(self):
        self.keepgoing = False

    def get_gaze_offset(self, cx, cy):
        return 640/2 - cx, 480/2 - cy
    
    def get_gaze_grad(self, offset_x, offset_y):
        ex = 0.2 * np.tanh(0.025*(offset_x))/ (1+self.Norm.pdf(offset_x)/self.scale*4)
        ey = 0.2 * np.tanh(0.025*(offset_y))/ (1+self.Norm.pdf(offset_x)/self.scale*4)
        return ex, ey

    def get_box_center(self, x1, y1, box_w, box_h):
        self.box_cener = (x1 + box_w/2, y1 + box_h/2)
        return x1 + box_w/2, y1 + box_h/2

    def track_obj(self, x,y,w,h):
        global torobo
        cx, cy = self.get_box_center(x, y, w, h)
        gaze_offset = self.get_gaze_offset(cx,cy)
        ex, ey = self.get_gaze_grad(*gaze_offset)
        hor, ver = get_cur_joints(torobo)[0][14:16]
        self.new_hor, self.new_ver = np.radians(hor)+ex, np.radians(ver)-ey

    def run(self):
        global imagebuff
        self.keepgoing = True
        while True and self.keepgoing:
            detections = self.recog.detect(imagebuff)
            if detections is not None:
                for det in detections:
                    for x1, y1, x2, y2, conf, cls_conf, cls_pred in detections:
                        if cls_pred == 0:
                            box_w = x2 - x1
                            box_h = y2 - y1
                            self.x1, self.y1, self.x2, self.y2 = x1, y1, x2, y2
                            x, y, w, h = x1, y1, box_w, box_h
                            self.track_obj(x, y, w, h)
                            
                            

class ExtForce(object):
    # SETS THE KINEMATIC MODEL PARAMETERS (BOTH IN HENDRY'S AND FACTORY FIRMWARE)

    class Mode(Enum):
        Teaching = 0
        Experiment = 1

    def runCommands(self, _commandList, _controller):
          for command in _commandList:
                torobo_easy_command.SendEasyCommandText(_controller, command)
                rospy.sleep(0.01)

    def registerParameters(self, tau_th, kp, sum_e_max, d, kr, theta_th):
            # left and right arm 
            for i in range(0,6):
                
                # constructing the commands for the left arm
                commandList = []				
                commandList.append("param " + str(i+1) + " fftauth " +       str(tau_th[i]))
                commandList.append("param " + str(i+1) + " ffkp " +          str(kp[i]))
                commandList.append("param " + str(i+1) + " ffsigmaemax " +   str(sum_e_max[i]))
                commandList.append("param " + str(i+1) + " ffdamping " +     str(d[i]))
                commandList.append("param " + str(i+1) + " ffkr " +          str(kr[i]))
                commandList.append("param " + str(i+1) + " softki " +        str(theta_th[i]))	


                # send to execution
                self.runCommands(commandList, "left_arm_controller")

            # right arm 
            for i in range(0,6):
                ii = i + 6 
                # constructing the commands for the right arm
                commandList = []
                commandList.append("param " + str(i+1) + " fftauth " +       str(tau_th[ii]))
                commandList.append("param " + str(i+1) + " ffkp " +          str(kp[ii]))
                commandList.append("param " + str(i+1) + " ffsigmaemax " +   str(sum_e_max[ii]))
                commandList.append("param " + str(i+1) + " ffdamping " +     str(d[ii]))
                commandList.append("param " + str(i+1) + " ffkr " +          str(kr[ii]))
                commandList.append("param " + str(i+1) + " softki " +        str(theta_th[ii]))	

                # send to execution
                self.runCommands(commandList, "right_arm_controller")
                            
            for i in range(0,4):
                ii = i + 12 
                # constructing the commands for the head-torso chain
                commandList = []				
                commandList.append("param " + str(i+1) + " fftauth " +       str(tau_th[ii]))
                commandList.append("param " + str(i+1) + " ffkp " +          str(kp[ii]))
                commandList.append("param " + str(i+1) + " ffsigmaemax " +   str(sum_e_max[ii]))
                commandList.append("param " + str(i+1) + " ffdamping " +     str(d[ii]))
                commandList.append("param " + str(i+1) + " ffkr " +          str(kr[ii]))
                commandList.append("param " + str(i+1) + " softki " +        str(theta_th[ii]))	

                # send to execution
                self.runCommands(commandList, "torso_head_controller")


    def __init__(self, _mode):	
        
        if _mode == ExtForce.Mode.Teaching:

            tau_th = [2.0, 2.0, 1.0, 1.0, 0.5, 0.5,            2.0, 2.0, 1.0, 1.0, 0.5, 0.5,      20.5, 20.5, 20.5, 20.5]
            kp = [0.1, 0.1, 0.05, 0.05, 0.1, 0.1,            0.1, 0.1, 0.05, 0.05, 0.1, 0.1,        0.0,0.0,0.0,0.0]
            sum_e_max = [200.0,200.0,100.0,100.0,50.0,50.0,  200.0,200.0,100.0,100.0,50.0,50.0,   200.0,200.0,50.0,50.0]
            d = [0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9]
            # kr = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
            kr = [0.1]*16
            kr[5] = 0.3
            kr[11] = 0.3
            theta_th = [1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57]
            theta_th = [i*1.63 for i in theta_th]

            self.registerParameters( tau_th, kp, sum_e_max, d, kr, theta_th)


        elif _mode == ExtForce.Mode.Experiment:

            tau_th = [12.0,12.0,5.0,5.0,1.5,1.5,12.0,12.0,5.0,5.0,1.5,1.5,200.0,200.0,200.0,200.0]
            kp = [0.025,0.025,0.05,0.05,0.1,0.1,0.025,0.025,0.05,0.05,0.1,0.1,0.025,0.025,0.05,0.05]
            sum_e_max = [200.0,200.0,100.0,100.0,50.0,50.0,200.0,200.0,100.0,100.0,50.0,50.0,200.0,200.0,50.0,50.0]
            d = [0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9]
            kr = [0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8]
            # kr = [0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9]
            theta_th = [1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57]
            theta_th = [i*1.63 for i in theta_th]

            self.registerParameters( tau_th, kp, sum_e_max, d, kr, theta_th)
            

    
    
def gostart(torobo):
    """ GO TO STARTING POSITION """
    idx = 4
    TIME = 5
    try:
        with HiddenPrints():
            servo_on(torobo)
            set_softness_override(torobo, 10.0)
            set_velocity_override(torobo, 10.0)
            with open('rad.pickle', 'rb') as f:
                RAD = pickle.load(f)

            left_arm_positions = RAD[idx][0:6]
            right_arm_positions = RAD[idx][6:12]
            torso_head_positions = RAD[idx][12:16]

            torobo.move(ToroboOperator.LEFT_ARM, positions=left_arm_positions, duration=TIME)
            torobo.move(ToroboOperator.RIGHT_ARM, positions=left_arm_positions, duration=TIME)
            torobo.move(ToroboOperator.TORSO_HEAD, positions=torso_head_positions, duration=TIME)
            rospy.sleep(TIME+0.1)
        print(colored('SUCCESS', 'white', 'on_green'))
    except:
        print('ERROR')
    

def setModeOverrides(torobo):
    """SET THE NEEDED KINEMATIC PARAMETERS AND MODES, OVERRIDES ON THE RIGHT JOINTS"""
    print('SETTING MODE: set_external_force_following_online_trajectory_control')
    with HiddenPrints():
        set_external_force_following_online_trajectory_control(torobo)
        torobo.set_control_mode(ToroboOperator.TORSO_HEAD, ['torso_head/joint_1',
                                                            'torso_head/joint_2'],
                                'position')
    print('SETTING DYNAMIC PARAMETERS')
    with HiddenPrints():
        time.sleep(1)
        ExtForce(ExtForce.Mode.Teaching)
        time.sleep(1)
    print('OVERRIDES SET')
    with HiddenPrints():
        set_softness_override(torobo, 10.0)
        set_velocity_override(torobo, 30.0)

plt.switch_backend('nbAgg')

bridge = CvBridge()
torobo = ToroboOperator()


In [2]:
gostart(torobo)
setModeOverrides(torobo)

SUCCESS
SETTING MODE: set_external_force_following_online_trajectory_control
SETTING DYNAMIC PARAMETERS
OVERRIDES SET


In [3]:
if not 'imagebuff' in locals():
    imagebuff = 0
    def image_callback(msg):
        global imagebuff
        imagebuff = bridge.imgmsg_to_cv2(msg, "rgb8") 
    rospy.Subscriber("/camera/color/image_raw", Image, image_callback)
    time.sleep(0.5)
    
tracker = Tracker()
tracker.start()

cv2.namedWindow('I see')
cv2.setWindowProperty("I see", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

while True:
    hor, ver = tracker.new_hor, tracker.new_ver
    torobo.move_joint_my (controller_id=ToroboOperator.TORSO_HEAD,
                          joint_ids = [0, 1, 2, 3],
                          positions=[0, np.radians(38.0), hor, ver],
                          velocities=None,
                          accelerations=None,
                          duration=0.2)
    try:
        center_coordinates = tuple(np.round(tracker.box_cener).astype('int'))
        radius = 20
        color = (255, 0, 0) 
        thickness = 2
        image = cv2.cvtColor(imagebuff, cv2.COLOR_RGB2BGR)
        image = cv2.rectangle(image, (int(tracker.x1), int(tracker.y1)), (int(tracker.x2), int(tracker.y2)), color, thickness) 
        cv2.imshow('I see', image)
        k = cv2.waitKey(3) & 0XFF
        if k== 27 :
            cv2.destroyWindow('I see')
            tracker.stop()
            break
    except:
        print('error')
        cv2.destroyWindow('I see')
        break
    
    time.sleep(0.2)

In [3]:
tracker.
(tuple(reversed(np.round(tracker.box_cener).astype('int'))))

(524.2368469238281, 229.44054412841797)


(230, 461)